<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/9g_Multivariate_and_Multi_step_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multivariate, Multi-step MLPs**



Multivariate, Multi-step models are a little harder than the previous MLPs that we looked at. <br>
Preparing the data and defining the inputs and output can be a bit more difficult. 

In this notebook we will look at two different models:<BR>
1. Multiple Input Multi-step Output
2. Multiple Parallel Input and Multi-step Output

**Multiple Input Multi-Step Output**

We can have a multivariate time series where the output series is separate but dependent on the the input series and mutiple time steps. 

For example:<BR>
A multivariate time series <br>
[input, input, output]<br>
[[10 15 25]<br>
 [20,25,45]<br>
 [30,35,65]<br>
 [ 40 45 85]<br>
 [ 50 55 105]<br>
 [ 60 65 125] <br>
 [ 70 75 145]<br>
 [ 80 85 165]<br>
 [ 90 95 185]]

If we use three steps of each of the two input series to predict two outputs<br>
For example: <br>
**input________output**<br>
10,15<br>
20,25________65<br>
30,35________85<br>

This split sequencess function is used to create the muti-step forecast for a dependent series

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out-1 
    # check if we are beyond the dataset 
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

**Import Libraries**

In [ ]:
from numpy import array
from numpy import hstack
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense

**Create the dataset**

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90]) 
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [ ]:
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

**Define the input and output steps**

In [ ]:
n_steps_in, n_steps_out = 3, 2

In [ ]:
X, y = split_sequences(dataset, n_steps_in, n_steps_out) 
print(X.shape, y.shape)

In [ ]:
for i in range(len(X)):
  print(X[i], y[i])

In [ ]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

**Create the model**

In [ ]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input)) 
model.add(Dense(n_steps_out)) 
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

**Train the model**

In [ ]:
model.fit(X, y, epochs=2000, verbose=0)

We are expecting the next two steps to be<br>
185 and 205

In [ ]:
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)